In [1]:
import torch
import torchaudio
import datasets
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import re

/sysadmin/wirawan/miniconda3/envs/bert2bert/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
# Preprocessing the datasets.
# We need to read the audio files as arrays
def speech_file_to_array_fn(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
    batch["sentence"] = batch["sentence"].replace('！ ', '')
    batch["sentence"] = batch["sentence"].replace('，', '')
    batch["sentence"] = batch["sentence"].replace('é', 'e')
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch

In [3]:
# Preprocessing the datasets.
# We need to read the aduio files as arrays
def evaluate(batch):
    inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_strings"] = processor.batch_decode(pred_ids)
    return batch

### Load the dataset

In [ ]:
def load_dataset_sundanese():
    root_dir = Path("/dataset/ASR/sundanese")
    url_sundanese_female = "https://www.openslr.org/resources/44/su_id_female.zip"
    url_sundanese_male = "https://www.openslr.org/resources/44/su_id_male.zip"
    data_dirs = [ root_dir/"su_id_female/wavs", root_dir/"su_id_male/wavs" ]
    filenames = [ root_dir/"su_id_female/line_index.tsv", root_dir/"su_id_male/line_index.tsv" ]
    
    if not (root_dir/"su_id_female").exists():
        !wget -P {root_dir} {url_sundanese_female}
        !unzip {root_dir}/su_id_female.zip -d {root_dir}
    if not (root_dir/"su_id_male").exists():    
        !wget -P {root_dir} {url_sundanese_male} 
        !unzip {root_dir}/su_id_male.zip -d {root_dir}
    
    dfs = []
    
    dfs.append(pd.read_csv(filenames[0], sep='\t\t', names=["path", "sentence"]))
    dfs.append(pd.read_csv(filenames[1], sep='\t\t', names=["path", "sentence"]))
    
    for i, dir in enumerate(data_dirs):
        dfs[i]["path"] = dfs[i].apply(lambda row: str(data_dirs[i]) + "/" + row + ".wav", axis=1)
    df = pd.concat(dfs)
    # df = df.sample(frac=1, random_state=1).reset_index(drop=True)
    dataset = Dataset.from_pandas(df)
    dataset = dataset.remove_columns('__index_level_0__')
    
    return dataset.train_test_split(test_size=0.1, seed=1)

In [4]:
%%capture

dataset = load_dataset_sundanese()
# We can also just load it from the disk created during training
# dataset = datasets.load_from_disk("dataset_sundanese")

DatasetDict({
    train: Dataset({
        features: ['path', 'sentence'],
        num_rows: 3791
    })
    test: Dataset({
        features: ['path', 'sentence'],
        num_rows: 422
    })
})

In [ ]:
dataset

In [5]:
test_dataset = dataset['test']
wer = load_metric("wer")

In [6]:
%%capture

model_name = "./wav2vec2-large-xlsr-sundanese"

processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name) 
model.to("cuda")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [7]:
#chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\'\”]'
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\'\”_\�]'
resampler = torchaudio.transforms.Resample(48_000, 16_000)
test_dataset = test_dataset.map(speech_file_to_array_fn)

Loading cached processed dataset at dataset_sundanese/test/cache-01a527eac70c10a5.arrow


In [8]:
result = test_dataset.map(evaluate, batched=True, batch_size=8)

print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))


WER: 6.190727
